In [41]:
from jupyter_plotly_dash import JupyterDash


import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "zxczxc"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))





#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'gm_logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'h2o'},
            {'label': 'Mountain Rescue', 'value': 'mtn'},
            {'label': 'Disaster Rescue', 'value': 'dis'},
            {'label': 'All', 'value': 'all'},
        ],
        value='all',
        labelStyle={'display': 'inline-block'}
    )

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.P("NATASHA WAS HERE X"),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
         children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',

                ),
            html.Div(
                id='map-id',
                className='col s12 m6',
                )
        ]),
    html.Br(),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Br(),
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
        if (filter_type == 'all'):
            df = pd.DataFrame.from_records(shelter.read({}))
        elif (filter_type == 'h2o'):
            df = pd.Dataframe.from_records(shelter.read({
                '$and': [ 
                    {'$or': [ 
                        {'breed':'Labrador Retriever Mix'},
                        {'breed':'Chesapeake Bay Retriever'},
                        {'breed':'Newfoundland'}
                    ]}, 
                    {'sex_upon_outcome':'Intact Female'},
                    {'age_upon_outcome_in_weeks':{'$gte':26, '$lte':156}}
                ]
            }))
        elif (filter_type == 'mtn'):
            df = pd.Dataframe.from_records(shelter.read({
                '$and': [ 
                    {'$or': [ 
                        {'breed':'German Shepherd'},
                        {'breed':'Alaskan Malamute'},
                        {'breed':'Old English Sheepdog'},
                        {'breed':'Siberian Husky'},
                        {'breed':'Rottweiler'}
                    ]}, 
                    {'sex_upon_outcome':'Intact Male'},
                    {'age_upon_outcome_in_weeks':{'$gte':26, '$lte':156}}
                ]
            }))
        elif (filter_type == 'dis'):
            df = pd.Dataframe(list(shelter.read({
                '$and': [ 
                    {'$or': [ 
                        {'breed':'Doberman Pinscher'},
                        {'breed':'German Sheperd'},
                        {'breed':'Golden Retriever'},
                        {'breed':'Bloodhound'},
                        {'breed':'Rottweiler'}
                    ]}, 
                    {'sex_upon_outcome':'Intact Male'},
                    {'age_upon_outcome_in_weeks':{'$gte':20, '$lte':300}}
                ]
            })))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed')
    return [
        dcc.Graph(            
            figure = fig
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # dff = viewDF.loc[rows]
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=15, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[0,'name'])
                ])
            ])
        ])
    ]

app
